In [1]:
import json

### Load Data

In [31]:
file_path = ''
with open(file_path) as df:
    data = json.load(df)

###  Get Scores

In [4]:
def get_scores(doc):
    scores = []
    clips = [clip.strip() for clip in doc['gpt4_eva'].split(':')]
    for clip in clips:
        if clip != '':
            if clip[0] == '0':
                scores.append(int(clip[0]))
            elif clip[:2].isdigit():
                if '100' in clip:
                    scores.append(100)
                else:
                    scores.append(int(clip[:2]))
            
    return scores

### Find outlier
In our results, these procedures have already been done, you can just skip them and directly see the results

In [6]:
# some generation results are not the normal format, which may cause truble when computing scores.

outlier_doc = []
scores = []
valid_keys = int()
for i in range(len(data)):
    score = get_scores(doc=data[i])
    scores.append(get_scores(doc=data[i]))
    if len(score) != valid_keys:
        outlier_doc.append(i)

In [ ]:
len(outlier_doc)

In [ ]:
eval_scores = []
for i in range(len(scores)):
    if len(scores[i]) != 0:
        score = scores[i]
        score_or = score[0]
        score_tr = max(score[1:])
        score_tuple = (score_or, score_tr)
        eval_scores.append(score_tuple)
    else:
        print(i)
        eval_scores.append((0,0))

In [13]:
for i in range(len(data)):
    data[i]['eval_scores'] = eval_scores[i]

### Calculate Final Scores

In [15]:
## This choose the original generation results and the best generation results
## The outcomes present in main results 

def compare_or_tr(data):
    score_or = 0
    score_tr = 0
    for doc in data:
        score_or += doc['eval_scores'][0]
        score_tr += doc['eval_scores'][1]
    score_or /= len(data)
    score_tr /= len(data)
    return score_or, score_tr

In [ ]:
compare_or_tr(data)

In [29]:
## This choose the original generation results and the average generation results for each set of parameters
## The outcomes present in ablation study results 

def all_aver_score(data):
    ### get scores and outlier doc
    outlier_doc = []
    scores = []
    for i in range(len(data)):
        score = get_scores(doc=data[i])
        scores.append(get_scores(doc=data[i]))
        if len(score) != 25:
            outlier_doc.append(i)
    
    ### get exact score for each kind of generation
    ## get keys
    keys = [key for key in data[0].keys()]
    keys = keys[3:-2]
    print(keys,len(keys))
    ## get score dict
    score_dict = {}
    for key in keys:
        score_dict[f'{key}'] = 0
        
    ### count all score
    keys = keys[:-1]
    length = len(data) - len(outlier_doc)
    
    for i in range(len(data)):
        if i not in outlier_doc:
            score_dict['decode_or'] += scores[i][0] / length
            for j in range(len(scores[i][1:])):
                score_dict[f'{keys[j-1]}'] += scores[i][j] / length
                           
    return score_dict

In [ ]:
all_aver_score(data)